In [48]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
import xarray as xr
from shapely.geometry import Polygon
import geojson
import os
os.chdir("/workspaces/aqqa-kg-creation-dev/")
from src.utils import clip_netcdf_to_bb

In [50]:
path_to_netcdf = "/mnt/data/raw/cams_euro_aq_reanalysis/2020/download_2020_01_all_variables/cams.eaq.vra.ENSa.co.l0.2020-01.nc"
ds = xr.open_dataset(path_to_netcdf)

In [51]:
bounding_box = [45.82, 50.65, 8.95, 17.22]
ds = clip_netcdf_to_bb(ds, bounding_box)

In [52]:
# Get the raster data and corresponding coordinates
data = ds.co.values
lon = ds.lon.values
lat = ds.lat.values

In [53]:
# Calculate the resolution of the grid cells
lon_resolution = lon[1] - lon[0]
lat_resolution = lat[1] - lat[0]

# Create empty lists to store polygons and values
index_polygon_dic = {}

In [54]:
# Iterate through each cell in the raster data
for row in range(len(lat)):
    for col in range(len(lon)):

        # Define the vertices of the polygon for the current cell
        lon_left = lon[col]
        lon_right = lon[col] + lon_resolution
        lat_bottom = lat[row]
        lat_top = lat[row] + lat_resolution

        # Create the polygon for the current cell
        polygon = Polygon([(lon_left, lat_bottom), (lon_right, lat_bottom),
                           (lon_right, lat_top), (lon_left, lat_top)])

        # Append the polygon and its value to the respective lists
        index_polygon_dic[(row, col)] = polygon

In [55]:
# save grid shapes as geosjaon
features = []
for index, geometry in index_polygon_dic.items():
    feature = geojson.Feature(geometry=geometry.__geo_interface__, properties={'index': index})
    features.append(feature)

# Create a FeatureCollection from the list of features
feature_collection = geojson.FeatureCollection(features)

# Export the FeatureCollection to a GeoJSON file
# Replace 'output_file.geojson' with the desired output file path
with open('/mnt/data/processed/ref_raster.geojson', 'w') as f:
    geojson.dump(feature_collection, f)